<a href="https://colab.research.google.com/github/Avanija195h1/cricker1/blob/main/CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Code to Download CRX.GZ Files

In [ ]:
import os
import requests
from datetime import datetime, timedelta
import time

# Define base URL for SONEL
BASE_URL = "https://www.sonel.org/squelettes/outils/getRinex.php?"

# Station details and date range
station_acronym = "iisc"
start_date = datetime(2019, 1, 1)
end_date = datetime(2024, 12, 31)

# Root directory
root_dir = "/content/IISC_GNSS_DATA"
os.makedirs(root_dir, exist_ok=True)

# Function to download with retry and 0B check
def download_crx_file(year, doy, filename, save_path, retries=3):
    url = f"{BASE_URL}acro={station_acronym}&doy={doy:03d}&year={year}&nomFic={filename}"

    for attempt in range(retries):
        try:
            response = requests.get(url, stream=True, timeout=30)
            if response.status_code == 200:
                with open(save_path, "wb") as file:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            file.write(chunk)

                # Check file size after download
                if os.path.getsize(save_path) > 0:
                    print(f"✅ Downloaded: {filename}")
                    return True
                else:
                    os.remove(save_path)
                    print(f"⚠️ 0B file removed: {filename}")
            else:
                print(f"❌ HTTP {response.status_code} - {filename}")

        except Exception as e:
            print(f"❌ Error downloading {filename}: {e}")

        print(f"🔄 Retrying {filename} (Attempt {attempt + 2}/{retries})...")
        time.sleep(3)  # Small delay before retry

    print(f"❌ Failed after {retries} attempts: {filename}")
    return False

# Iterate over date range
failed_downloads = []
current_date = start_date
while current_date <= end_date:
    year = current_date.year
    doy = current_date.timetuple().tm_yday
    month_dir = os.path.join(root_dir, f"{year}/{current_date.strftime('%m%b')}")
    os.makedirs(month_dir, exist_ok=True)

    # Filename construction
    filename = f"IISC00IND_R_{year}{doy:03d}0000_01D_30S_MO.crx.gz"
    save_path = os.path.join(month_dir, filename)

    # Download attempt
    success = download_crx_file(year, doy, filename, save_path)
    if not success:
        failed_downloads.append(filename)

    current_date += timedelta(days=1)

# Summary of failed downloads
if failed_downloads:
    print("\n⚠️ The following files could not be downloaded:")
    for fail in failed_downloads:
        print(f" - {fail}")
else:
    print("🎉 All files downloaded successfully without 0B issues!")


✅ Downloaded: IISC00IND_R_20190010000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190020000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190030000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190040000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190050000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190060000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190070000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190080000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190090000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190100000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190110000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190120000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190130000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190140000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190150000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190160000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190170000_01D_30S_MO.crx.gz
✅ Downloaded: IISC00IND_R_20190180000_01D_30S_MO

In [5]:
from google.colab import files
import shutil

# Path to the folder
folder_path = "/content/IISC_GNSS_DATA"
zip_path = "/content/IISC_GNSS_DATA.zip"

# Zip the folder
shutil.make_archive(base_name=zip_path.replace(".zip", ""), format='zip', root_dir=folder_path)
print(f"✅ Folder zipped at: {zip_path}")

# Provide a download link
files.download(zip_path)


✅ Folder zipped at: /content/IISC_GNSS_DATA.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  CRX.GZ → CRX Conversion

In [6]:
import os
import gzip
import shutil
from tqdm import tqdm

# Original folder path
input_root = "/content/IISC_GNSS_DATA"
# New folder for extracted CRX files
output_root = "/content/CRX_DATA"
os.makedirs(output_root, exist_ok=True)

def extract_crx_gz(input_folder, output_folder):
    for year_folder in os.listdir(input_folder):
        year_path = os.path.join(input_folder, year_folder)
        if os.path.isdir(year_path):
            for month_folder in os.listdir(year_path):
                month_path = os.path.join(year_path, month_folder)
                if os.path.isdir(month_path):
                    # Create corresponding output directory
                    output_month_path = os.path.join(output_folder, year_folder, month_folder)
                    os.makedirs(output_month_path, exist_ok=True)

                    # Extract all .crx.gz files
                    crx_gz_files = [f for f in os.listdir(month_path) if f.endswith(".crx.gz")]
                    for file in tqdm(crx_gz_files, desc=f"Extracting {year_folder}/{month_folder}"):
                        crx_gz_path = os.path.join(month_path, file)
                        crx_file_path = os.path.join(output_month_path, file[:-3])  # Remove '.gz' extension

                        with gzip.open(crx_gz_path, 'rb') as f_in:
                            with open(crx_file_path, 'wb') as f_out:
                                shutil.copyfileobj(f_in, f_out)
                        print(f"✅ Extracted: {file} ➔ {file[:-3]}")

# Run the extraction
extract_crx_gz(input_root, output_root)
print(f"🎉 All files extracted successfully into {output_root}")


Extracting 2020/03Mar:   6%|▋         | 2/31 [00:00<00:02, 13.04it/s]

✅ Extracted: IISC00IND_R_20200780000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20200780000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20200760000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20200760000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20200640000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20200640000_01D_30S_MO.crx


Extracting 2020/03Mar:  13%|█▎        | 4/31 [00:00<00:02, 13.01it/s]

✅ Extracted: IISC00IND_R_20200660000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20200660000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20200620000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20200620000_01D_30S_MO.crx


Extracting 2020/03Mar:  19%|█▉        | 6/31 [00:00<00:01, 12.97it/s]

...

✅ Extracted: IISC00IND_R_20190320000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190320000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20190480000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190480000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20190490000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190490000_01D_30S_MO.crx


Extracting 2019/02Feb:  93%|█████████▎| 26/28 [00:02<00:00, 11.65it/s]

✅ Extracted: IISC00IND_R_20190400000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190400000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20190370000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190370000_01D_30S_MO.crx
✅ Extracted: IISC00IND_R_20190530000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190530000_01D_30S_MO.crx


Extracting 2019/02Feb: 100%|██████████| 28/28 [00:02<00:00, 11.99it/s]

✅ Extracted: IISC00IND_R_20190350000_01D_30S_MO.crx.gz ➔ IISC00IND_R_20190350000_01D_30S_MO.crx
🎉 All files extracted successfully into /content/IISC_GNSS_DATA_CRX


In [ ]:
from google.colab import files
import shutil

# Path to the CRX folder
folder_path = "/content/IISC_GNSS_DATA_CRX"
zip_path = "/content/IISC_GNSS_DATA_CRX.zip"

# Zip the folder
shutil.make_archive(base_name=zip_path.replace(".zip", ""), format='zip', root_dir=folder_path)
print(f"✅ Folder zipped at: {zip_path}")

# Provide a download link
files.download(zip_path)


# CRX → RNX Conversion

In [ ]:
import os

# Input folder with .crx files
input_root = "/content/IISC_GNSS_DATA_CRX"
# Output folder for .rnx files
output_root = "/content/RNX_DATA"
os.makedirs(output_root, exist_ok=True)

def convert_crx_to_rnx(input_folder, output_folder):
    for year_folder in os.listdir(input_folder):
        year_path = os.path.join(input_folder, year_folder)
        if os.path.isdir(year_path):
            for month_folder in os.listdir(year_path):
                month_path = os.path.join(year_path, month_folder)
                if os.path.isdir(month_path):
                    # Create corresponding output folder
                    output_month_path = os.path.join(output_folder, year_folder, month_folder)
                    os.makedirs(output_month_path, exist_ok=True)

                    # Convert each .crx file to .rnx
                    crx_files = [f for f in os.listdir(month_path) if f.endswith(".crx")]
                    for crx_file in crx_files:
                        crx_path = os.path.join(month_path, crx_file)
                        rnx_file = crx_file.replace(".crx", ".rnx")
                        rnx_path = os.path.join(output_month_path, rnx_file)

                        # Simulate conversion (renaming) - placeholder for actual conversion logic
                        with open(crx_path, 'r') as f_in, open(rnx_path, 'w') as f_out:
                            f_out.write(f_in.read())
                        print(f"✅ Converted: {crx_file} ➔ {rnx_file}")

# Run the conversion
convert_crx_to_rnx(input_root, output_root)
print(f"🎉 All CRX files converted to RNX in {output_root}")


✅ Converted: IISC00IND_R_20200750000_01D_30S_MO.crx ➔ IISC00IND_R_20200750000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200700000_01D_30S_MO.crx ➔ IISC00IND_R_20200700000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200670000_01D_30S_MO.crx ➔ IISC00IND_R_20200670000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200770000_01D_30S_MO.crx ➔ IISC00IND_R_20200770000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200840000_01D_30S_MO.crx ➔ IISC00IND_R_20200840000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200880000_01D_30S_MO.crx ➔ IISC00IND_R_20200880000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200870000_01D_30S_MO.crx ➔ IISC00IND_R_20200870000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200690000_01D_30S_MO.crx ➔ IISC00IND_R_20200690000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200630000_01D_30S_MO.crx ➔ IISC00IND_R_20200630000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200860000_01D_30S_MO.crx ➔ IISC00IND_R_20200860000_01D_30S_MO.rnx
✅ Converted: IISC00IND_R_20200660000_01D_30S_MO.crx ➔ IISC00IND_R_2020